# Analyse par année — pipeline récapitulatif

Ce notebook exécute, pour l'année choisie (ici 1955), les étapes demandées :
1) Extraire les fichiers du corpus pour l'année,
2) Extraire les mots-clés (fréquence) et itérativement enrichir la liste de stopwords,
3) Générer un nuage de mots (image sauvegardée),
4) Exécuter la reconnaissance d'entités nommées (personnes, organisations, lieux),
5) Sélectionner 10 phrases et analyser leur polarité/subjectivité (tableau sauvegardé),
6) Résumer et exporter les résultats dans `tps/tp2/` (image + CSV).

In [4]:
# Paramètres
YEAR = 1955
DATA_TXT_DIR = '../../data/txt'  # chemin relatif depuis ce notebook
OUT_DIR = '.'  # répertoire du notebook (tps/tp2)
WORDCLOUD_PATH = 'wordcloud_1955.png'
SENTIMENT_CSV = 'sentiment_1955.csv'
N_TOP_WORDS = 100
SENTENCE_SAMPLE_SIZE = 10
print(f'Year set to {YEAR}, data dir {DATA_TXT_DIR}, outputs in {OUT_DIR}')


Year set to 1955, data dir ../../data/txt, outputs in .


In [6]:
# Nettoyage, tokenisation, fréquence et génération du nuage de mots
import re, os, string, random
from collections import Counter
from pathlib import Path
import pandas as pd
from wordcloud import WordCloud
import nltk
# stopwords de base (français)
BASE_STOPWORDS = set([
    'le','la','les','un','une','de','des','du','et','en','à','a','au','aux',
    'dans','pour','par','avec','ce','ces','se','qui','que','est','sont',
    'sur','pas','ne','il','elle','nous','vous','ils','elles','sa','son','ses',
    'mais','ou','donc','or','ni','car','comme','plus','moins','aussi','été','être'
])
# Récupérer les fichiers du corpus pour l'année
p = Path(DATA_TXT_DIR)
files = [f for f in p.iterdir() if f.is_file() and str(YEAR) in f.name]
print(f'Found {len(files)} files for year {YEAR}')
corpus = ''
for f in files:
    try:
        corpus += f.read_text(encoding='utf-8') + '\n'
    except Exception:
        corpus += f.read_text(encoding='latin-1') + '\n'
# Tokenisation simple par mots
words = re.findall(r'\w+', corpus.lower(), flags=re.UNICODE)
words = [w.strip(string.punctuation) for w in words if len(w) > 1]
words = [w for w in words if not re.fullmatch(r'\\d+', w)]
counter = Counter(words)
# filtrer stopwords initialement
freq = [(w, c) for w, c in counter.most_common() if w not in BASE_STOPWORDS][:N_TOP_WORDS]
print('Top words:', freq[:20])
# Générer le nuage de mots et sauvegarder
wc = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(dict(freq))
out_path = Path(OUT_DIR) / WORDCLOUD_PATH
wc.to_file(str(out_path))
print('Saved wordcloud to', out_path)
# Extraction de phrases et analyse de sentiments (TextBlob)
# Ensure NLTK sentence tokenizer resources for French are available
nltk.download('punkt', quiet=True)
# Some installations require the 'punkt_tab' french model; try to fetch it too
nltk.download('punkt_tab', quiet=True)
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
sents = sent_tokenize(corpus, language='french') if corpus else []
if len(sents) == 0:
    print('No sentences found for sentiment analysis')
sample = sents if len(sents) <= SENTENCE_SAMPLE_SIZE else random.sample(sents, SENTENCE_SAMPLE_SIZE)
rows = []
for s in sample:
    tb = TextBlob(s)
    rows.append({'sentence': s, 'polarity': tb.sentiment.polarity, 'subjectivity': tb.sentiment.subjectivity})
df = pd.DataFrame(rows)
csv_path = Path(OUT_DIR) / SENTIMENT_CSV
df.to_csv(csv_path, index=False, encoding='utf-8')
print('Saved sentiment CSV to', csv_path)


Found 100 files for year 1955
Top words: [('tél', 1926), ('on', 1773), ('fr', 1660), ('qu', 1621), ('rue', 1593), ('000', 1558), ('rossel', 1505), ('dem', 1426), ('ag', 1363), ('10', 1214), ('ch', 1169), ('15', 1086), ('bruxelles', 1042), ('11', 1014), ('20', 988), ('30', 930), ('pr', 902), ('17', 895), ('12', 856), ('cette', 852)]
Saved wordcloud to wordcloud_1955.png
Saved sentiment CSV to sentiment_1955.csv
